In [12]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Convolution2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD

In [13]:
img_width, img_height = 300, 300

epochs = 15
batch_size = 64

In [14]:
import pandas
import sys
import os
import cv2, numpy as np
def load_data(data_path, input_width,input_height):
    print('Loading data... ', end='')
    columns = ['data','label']
    df = pandas.DataFrame(columns=columns)
    for root, subFolder, files in os.walk(data_path):
        for item in files:
            if item.endswith(".png") or item.endswith(".jpg") :
                fileNamePath = str(os.path.join(root,item))
                im = cv2.imread(fileNamePath)
                im = cv2.resize(im, (input_width,input_height))
                imgArray = np.asarray(im)
                
                classStr = ''.join(fileNamePath.split("\\")[-2:-1])
                if classStr == 'ar':
                    classStr = 0
                else:
                    classStr = 1
                
                df.loc[len(df)] = [imgArray, classStr]
    data = np.array(df['data'].tolist())
    label = np.array(df['label'].tolist())
    print('%d data loaded.'%len(df))
    return data,label

In [15]:
import random

XData,YData = load_data('data_augmented', img_width,img_height)

s = np.arange(XData.shape[0])
np.random.shuffle(s) 
n_train = round(XData.shape[0]*0.7)
idx_train = s[:n_train]
idx_val = s[n_train:]

XTrain,YTrain = XData[idx_train],YData[idx_train]
XVal,YVal = XData[idx_val],YData[idx_val]

Loading data... 501 data loaded.


In [16]:
from keras import optimizers

input_shape = (img_height,img_width,3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

Nadam = optimizers.Nadam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.003)
model.compile(loss='binary_crossentropy', optimizer=Nadam, metrics=['accuracy'])

In [ ]:
model.fit(XTrain,YTrain, epochs=epochs, batch_size=batch_size, validation_data=(XVal,YVal))
model.save('weights_cnn_fit_aug.h5')

Train on 351 samples, validate on 150 samples
Epoch 1/15
351/351 [==============================] - 19s 53ms/step - loss: 5.4506 - acc: 0.5755 - val_loss: 7.3148 - val_acc: 0.5067
Epoch 2/15
320/351 [==========================>...] - ETA: 1s - loss: 5.5499 - acc: 0.5906